In [ ]:
# redis 测试
import redis
from wxpy import *
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
 
# Create a new chat bot named Charlie
chatbot = ChatBot('联通机器人')
 
trainer = ListTrainer(chatbot)
 
trainer.train([
    "联通","我爱联通公司","你呢"
])
 

# init redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,decode_responses=True)
# init bot
bot = Bot()

# info store in redis as list
def group_info_store(info):
    r.rpush('group_info',info)
# export group info
def group_info_export():
    size=r.llen("group_info") 
    for i in range(0,size):
        print(r.lindex("group_info",i))
# delete group info
def remove_group_info():
    size=r.llen("group_info") 
    for i in range(0,size):
        r.brpop('group_info')

# deal info with group   
@bot.register(Group,TEXT)
def auto_accept_friends(msg):
#     msg.chat.send(msg.text)
    username=msg.member.name
    info=msg.text
    user_info=str(username)+":"+str(info)
    print(info)
    print(chatbot.get_response(info))
    msg.chat.send(chatbot.get_response(info))
    group_info_store(user_info)
    
    if(info=='导出聊天记录'):
        print('----导出中----')
        group_info_export()
        print('----导出完成----')
    elif(info=='删除聊天记录'):
        print('----删除中----')
        remove_group_info()
        print('----删除完成----')
        msg.chat.send('删除完成')
@bot.register(Friend,TEXT)
def auto_accept_friends(msg):
    msg.chat.send(msg.text)


In [81]:
# chatterbot 机器人回答
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,decode_responses=True)
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
# Create a new chat bot named 联通机器人
chatbot = ChatBot('联通机器人')
# init robot
trainer = ListTrainer(chatbot)


# add train data
def insert_data(info):
    r.rpush('data',info)
    train_data()
# get all data
def get_all_data():
    size=r.llen("data")
    train_data=[]
    for i in range(0,size):
        train_data.append(r.lindex("data",i))
    return train_data
    
# use robot train data
def train_data():
    data=get_all_data()
    trainer.train(data)
    return trainer


In [156]:
# map 匹配回答
import redis
from wxpy import *
import redis
# 初始化redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,decode_responses=True)
# 初始化机器人
bot = Bot()
bot.enable_puid('wxpy_puid.pkl')
# 插入题库
# 插入方式，问题：你好，请问你叫什么？  答案：我叫张润民
def take_Q_S(info):
    first_index=pattern.search(info).span(0)[1]+1
    tempxx=(info[first_index:])
    second_index=tempxx.find('答')+first_index
    question=info[first_index:second_index].strip()
    answer=info[second_index+3:].strip()
    print(question,answer)
    return question,answer
# 数据插入map
def insert_map_data(question,answer):
    r.hset("QandA", question, answer)
    print('问题已经录入成功')
# 获取答案
def get_answer(question):
    return r.hget("QandA", question)
# 删除数据
def delete_data(info):
    r.hdel("QandA",info)
    print('删除成功')

# info store in redis as list
def group_info_store(info):
    r.rpush('group_info',info)
# export group info
def group_info_export():
    size=r.llen("group_info") 
    for i in range(0,size):
        print(r.lindex("group_info",i))
# delete group info
def remove_group_info():
    size=r.llen("group_info") 
    for i in range(0,size):
        r.brpop('group_info')

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 我是个无情的机器人


In [164]:
# deal info with group   
@bot.register(Group,TEXT)
def auto_accept_friends(msg):
#     msg.chat.send(msg.text)
    username=msg.member.name
    info=msg.text
#     print(msg.member.puid)
#     print(type(msg.member.puid))
    user_info=str(username)+":"+str(info)
    group_info_store(user_info)
    a=get_answer(info[1:])
    
    if (info.strip()[0] in ['~','～']) and a==None:
        msg.chat.send('别问啦，我还需要学习呢')
#         print(bot.self.name)
        myself_name=bot.self.name
        myself_info=str(myself_name)+":"+str('别问啦，我还需要学习呢')
        group_info_store(myself_info)
    elif(info.strip()[0] in ['~','～']):
        msg.chat.send(a)
        myself_name=bot.self.name
        myself_info=str(myself_name)+":"+str(a)
        group_info_store(myself_info)
    else:
        raise Exception("系统出问题了")
    if (info.strip()[0] in ['~','～']) and '导出聊天记录' in info:
        print('-------------导出中---------------')
        group_info_export()
        print('-------------导出完成-------------')

-------------导出中---------------
ZRM:11
ZRM:～1
ZRM:~导出聊天记录
ZRM:～～
我是个无情的机器人:别问啦，我还需要学习呢
ZRM:~导出聊天记录
我是个无情的机器人:别问啦，我还需要学习呢
-------------导出完成-------------


In [254]:
import distance
def cal_diff_setence_1(info,redis_list):
    marks=dict()
    for i in redis_list: 
        score=distance.levenshtein(info, i)
        marks[i]=score
    print(marks)
    return (min(marks, key=marks.get))

In [258]:
import gensim
import jieba
import numpy as np
from scipy.linalg import norm